In [43]:
import requests
from bs4 import BeautifulSoup
!pip install tqdm
from tqdm import tqdm

# データを格納するリスト
jobs = []

# ページをループ
for page in tqdm(range(1, 427), desc="ページ処理中"):
    url = f'https://willof.jp/techcareer/engineer/search?page={page}'
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')

    # 各求人情報を取得
    for job in soup.select('li.inner-list'):
        company_name = job.select_one('.company-name')
        job_title = job.select_one('.article-name a')
        salary = job.select_one('.reward-amount')
        industry = job.select('div.work-info-data')
        detail_url = job.select_one('.read-more-button')

        if company_name and job_title and salary and industry and detail_url:
            job_data = {
                'company_name': company_name.text.strip(),
                'job_title': job_title.text.strip(),
                'salary': salary.text.strip(),
                'industry': industry[1].text.strip().replace('\n', '').replace(' ', '') if len(industry) > 1 else 'N/A',
                'detail_url': detail_url['href']
            }

            # 詳細URLにアクセスして追加データを取得
            detail_response = requests.get(detail_url['href'])
            detail_soup = BeautifulSoup(detail_response.content, 'html.parser')

            # 代表者、本社在地、企業URLを取得
            representative = detail_soup.select_one('.article-list:-soup-contains("代表者") .article-inner')
            headquarters = detail_soup.select_one('.article-list:-soup-contains("本社在地") .article-inner')
            company_url = detail_soup.select_one('.article-list:-soup-contains("企業URL") .link-underline')

            if representative and headquarters and company_url:
                job_data['representative'] = representative.text.strip()
                job_data['headquarters'] = headquarters.text.strip()
                job_data['company_url'] = company_url['href']

            jobs.append(job_data)

# 結果を表示
df = pd.DataFrame(jobs)
df.to_csv('wot-scraping.csv', index=False, encoding='utf-8')

ページ処理中: 100%|██████████| 426/426 [1:04:05<00:00,  9.03s/it]


,company_name,job_title,salary,industry,detail_url,representative,headquarters,company_url
0,株式会社サイバーエージェント,【役員直下プロジェクト】生成AIで全社の業務効率化と事業創出を推進するエンジニア,500万円 - 800万円,ゲーム、広告,https://willof.jp/techcareer/detail/21689,藤田 晋,〒150-0042\n東京都渋谷区宇田川町40番1号 Abema Towers,https://www.cyberagent.co.jp/
1,株式会社サイバーエージェント,【インフラエンジニア】小売DXシステムの設計・開発/アーキテクチャ設計/機能改善,600万円 - 1200万円,ゲーム、広告,https://willof.jp/techcareer/detail/13659,藤田 晋,〒150-0042\n東京都渋谷区宇田川町40番1号 Abema Towers,https://www.cyberagent.co.jp/
2,株式会社サイバーエージェント,【iOS・Androidエンジニア】小売DX/アプリ・EC開発/データグロース,600万円 - 1200万円,ゲーム、広告,https://willof.jp/techcareer/detail/13656,藤田 晋,〒150-0042\n東京都渋谷区宇田川町40番1号 Abema Towers,https://www.cyberagent.co.jp/
3,株式会社サイバーエージェント,【機械学習エンジニア】AI事業本部 / オープンポジション / 大規模データ活用,600万円 - 1200万円,ゲーム、広告,https://willof.jp/techcareer/detail/13648,藤田 晋,〒150-0042\n東京都渋谷区宇田川町40番1号 Abema Towers,https://www.cyberagent.co.jp/
4,株式会社サイバーエージェント,【データサイエンティスト】AI事業本部/オープンポジション/広告配信プロダクト,600万円 - 1200万円,ゲーム、広告,https://willof.jp/techcareer/detail/13649,藤田 晋,〒150-0042\n東京都渋谷区宇田川町40番1号 Abema Towers,https://www.cyberagent.co.jp/
...,...,...,...,...,...,...,...,...
3753,MKIテクノロジーズ株式会社,システムエンジニア（サービスデスク3部）,430万円 - 730万円,N/A,https://willof.jp/techcareer/detail/18789,山本 篤,〒164-0003 東京都中野区東中野2-7-14,https://www.mkitec.co.jp/company/outline.html
3754,MKIテクノロジーズ株式会社,【大阪】サーバエンジニア（ITインフラサービス2部）,430万円 - 730万円,N/A,https://willof.jp/techcareer/detail/18788,山本 篤,〒164-0003 東京都中野区東中野2-7-14,https://www.mkitec.co.jp/company/outline.html
3755,MKIテクノロジーズ株式会社,サーバエンジニア（サービスデスク2部）,430万円 - 730万円,N/A,https://willof.jp/techcareer/detail/18787,山本 篤,〒164-0003 東京都中野区東中野2-7-14,https://www.mkitec.co.jp/company/outline.html
3756,MKIテクノロジーズ株式会社,音声系 システムエンジニア（アカウントサービス部）,430万円 - 730万円,N/A,https://willof.jp/techcareer/detail/18786,山本 篤,〒164-0003 東京都中野区東中野2-7-14,https://www.mkitec.co.jp/company/outline.html
